# Example Analysis

This notebook provides some example analysis that can be done when combining the supply, demand, and regional datasets from the previous notebooks.

## Calculate per capita ICU capacity for SF vs Bay Area counties

This calculuation will compute per capita numbers for ICU capacity for San Francisco and counties in the Bay Area.

In [ ]:
import os

import pandas as pd
from geopandas import gpd

from covidcaremap.data import published_data_path, processed_data_path
from covidcaremap.geo import sum_per_region

Read in the facility CCM data and county polygons.

In [ ]:
ccm_facility_data_gpd = gpd.read_file(published_data_path(
    'us_healthcare_capacity-facility-CovidCareMap.geojson'))

county_gdf = gpd.read_file(processed_data_path('us_counties_with_pop.geojson'))

Filter the counties down to the regions of interest.

In [ ]:
counties_of_interest = ['Alamedac County',
                        'Marin County', 
                        'Sonoma County', 
                        'Napa County', 
                        'Solano County', 
                        'Contra Costa County',
                        'Santa Cruz County']

filtered_county_gdf = county_gdf
filtered_county_gdf['Region Name'] = filtered_county_gdf['County Name'] + ' County'
filtered_county_gdf = filtered_county_gdf[filtered_county_gdf['Region Name'].isin(counties_of_interest)]
filtered_county_gdf = filtered_county_gdf[['Region Name', 'Population', 'geometry']]
filtered_county_gdf

Fetch the SF polygon and set population and a region id to match the county level data, and then merge it in with the county data.

In [ ]:
# Get san francisco GeoJSON        
sf_neighborhoods_gdf = gpd.read_file('https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/san-francisco.geojson')

sf_neighborhoods_gdf['region_id'] = 1

# This data is a set of neighborhood polygons.
# Use the 'dissolve' method to merge them into a single polygon.
sf_gdf = sf_neighborhoods_gdf.dissolve(by='region_id')[['geometry']]

# Source: 2017 Census data
sf_gdf['Region Name'] = 'San Francisco'
sf_gdf['Population'] = 884363

counties_and_sf_gdf = pd.concat([filtered_county_gdf,sf_gdf])


Run the `sum_per_region` which takes facility-level data and creates a summary DF for the regions.

In [ ]:
result_gdf = sum_per_region(ccm_facility_data_gpd,
                   counties_and_sf_gdf,
                   groupby_columns='Region Name',
                   region_id_column='Region Name',
                   population_columns={'People': 'Population'})
result_df = result_gdf[['Region Name',
                        'Staffed ICU Beds',
                        'Staffed All Beds',
                        'Licensed All Beds',
                        'All Bed Occupancy Rate',
                        'ICU Bed Occupancy Rate',
                        'Staffed ICU Beds [Per 1000 People]',
                        'Staffed All Beds [Per 1000 People]',
                        'Licensed All Beds [Per 1000 People]']]
result_df

## Comparing CHIME and IHME forecasts

Here we make a comparison between the CHIME and IHME forecasts.

In [ ]:
from datetime import datetime, timedelta

from covidcaremap.data import read_us_states_gdf 
from covidcaremap.cases import get_nytimes_cases_by_state
import covidcaremap.chime as ccm_chime
from covidcaremap.ihme import IHME

Take the cases from 2020-03-25 as a starting point and run chime for 30 days.

In [ ]:
states_df = read_us_states_gdf()
states_df

In [ ]:
cases_by_state = get_nytimes_cases_by_state()
cases_by_state = cases_by_state[cases_by_state['date'] == '2020-03-25'].merge(
    states_df, left_on='state', right_on='State Name'
)

Run chime against the states. Chime seems to need to run for more days than you want, so set to 40

In [ ]:
chime_predictions = ccm_chime.get_regional_predictions(
    cases_by_state,
    'State Name',
    cases_column='cases',
    num_days=40
)

In [ ]:
chime_on_date = chime_predictions[chime_predictions['day'] == 30]
chime_on_date

Load the IHME data and filter to our compare date

In [ ]:
ihme_df = get_ihme_forecast()

In [ ]:
target_date = (datetime(2020,3,25) + timedelta(days=30)).strftime('%Y-%m-%d')
ihme_on_date = ihme_df[ihme_df['date_reported'] == target_date]
ihme_on_date

Merge the ihme and chime data, then select some comparable columns.

In [ ]:
ihme_chime = ihme_on_date.rename(columns={'location_name': 'State Name'}) \
                         .merge(chime_on_date, on='State Name')

ihme_chime_subset = ihme_chime[[
    'State Name',
    'admis_lower',
    'admis_mean',
    'admis_upper',
    'hospitalized_admitted'
]]
ihme_chime_subset

## Comparing HGHI and CovidCareMap.org occupancy rates

Left as an exercise to the reader!

## Calculate ICU capacity gap metric

Another exercise!

Calculate the difference between available ICU beds (Staffed ICU Beds minus those occupied, determined by the ICU Bed Occupancy Rate) and `ICUbed_mean` projections from IHME
per state. Use this to determine if there is a date at which the projected ICU demands
outstrip the capacity.